# Flutter of the Heave-Pitch Typical Section with Quasi-Steady Aerodynamics

Welcome back! 👋

In our last notebook, we looked at a the typical section that could only move up and down (heave motion). We found that while it had some interesting damping behavior, it generally refused to destroy itself dynamically. In other words, it was always stable.

However, real wings don't just bend; they also twist. And this is a key ingredient for the phenomenon of bending-torsion flutter. For this reason, in this notebook we are going to combine heave $(h)$ and pitch $(\theta)$ motions to create a typical section model with two degrees of freedom (2-DOF).

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand how heave and pitch motions couple mechanically
2. Derive the equations of motion for the coupled system
3. Apply quasi-steady aerodynamics to the 2-DOF system
4. Formulate and solve the flutter eigenvalue problem
5. Interpret V-g diagrams when instability occurs
6. Understand the physical mechanism of bending-torsion flutter
7. Explore how design parameters affect flutter speed

_Let's get started!_ 🚀

In [ ]:
# We start by importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Enable interactive plots - remember to change "widget" to "inline" if running in Google Colab
%matplotlib widget

## The Physical Setup: Heave and Pitch Degrees of Freedom and the Inertial Coupling

Look at our new configuration of the typical section below. This time, it's supported by two springs:
- $k_h$: The bending stiffness (resists vertical motion).
- $k_\alpha$: The torsional stiffness (resists twisting).

Consequently, we have two degrees of freedom:
- $h$ (Heave): Vertical displacement (positive up).
- $\theta$ (Pitch): Angular displacement (positive nose-up).

Additionally, we have three notable points on the section:
- **Aerodynamic Center (AC):** The point where the aerodynamic forces are assumed to act.
    - This point is located at the quarter-chord of the airfoil. Since in aeroelasticity we usually work with the half-chord $b=c/2$, we will say that the AC is located at $b/2$ from the leading edge.
- **Elastic Axis (EA):** The point about which the section twists.
    - We denote the distance from the leading edge to the EA as $\left(1 + e\right)b$. Essentially, $e$ represents the nondimensional offset of the EA from the mid-chord.
- **Center of Gravity (CG):** The point where we can consider the mass to be concentrated.
    - We denote the distance from the leading edge to the CG as $\left(1 + a\right)b$. Here, $a$ represents the nondimensional offset of the CG from the mid-chord.

We also have the following inertial properties:
- $m$: Mass of the section.
- $I_\mathrm{CG}$: Mass moment of inertia about the CG.

Finally, we define two reference frames:
- An inertial frame $(\hat{\boldsymbol{i}}_1, \hat{\boldsymbol{i}}_2)$ fixed to the ground.
- A body-fixed frame $(\hat{\boldsymbol{b}}_1, \hat{\boldsymbol{b}}_2)$ that moves with the section.

<figure>
<img src="../figures/03_typical_section_heave_pitch_dof.svg" style="width:100%">
<figcaption align = "center"> Typical section with heave and pitch degrees of freedom. </figcaption>
</figure>

Stop and Think: If you move the CG up and down, does the section twist? And if you twist the section, does the CG move in the heave direction?

The idea is that if the CG is located exactly at the EA, the two motions are independent. However, if the CG is offset by a distance from the EA, the two motions are coupled through inertial effects. This mechanical coupling is another type of coupling that we need to take into account in addition to the aeroelastic coupling that links the aerodynamic forces with the structural motion.

We are now going to derive the equations of motion for our system and see how these couplings manifest mathematically.

## Derivation of the Equations of Motion

For a system with coupled degrees of freedom like ours, deriving equations of motion using force and moment equilibrium can become algebraically messy. We'd need to carefully track forces at multiple points and resolve them through the coupling geometry. Lagrange's equations offer a more systematic approach: we express the system's energy (kinetic and potential) and let the calculus machinery handle the derivatives. This energy-based method automatically accounts for all coupling terms and is particularly elegant for systems where constraints and multiple DOFs interact.

The approach is simple: we define your generalized coordinates $\boldsymbol{q}$ (in our case, $h$ and $\theta$), write down the kinetic energy $T$ and potential energy $V$, compute the Lagrangian $\mathscr{L} = T - V$, and then apply Lagrange's equations to get the equations of motion. Let's see how this works for our typical section.

We start from Lagrange's equations:

$$\frac{\mathrm{d}}{\mathrm{d}t}\left(\frac{\partial \mathscr{L}}{\partial \dot{\boldsymbol{q}}}\right) - \frac{\partial \mathscr{L}}{\partial \boldsymbol{q}} = \boldsymbol{Q},$$

where $\boldsymbol{Q}$ are the non-conservative generalized forces acting on our system.

The generalized coordinates for our system are:

$$\boldsymbol{q} = \begin{bmatrix} h \\ \theta \end{bmatrix}.$$

The potential energy of the system is given by:

$$V = \frac{1}{2}k_h h^2 + \frac{1}{2}k_\theta \theta^2.$$

The total kinetic energy of the system is given by the sum of the translational and rotational kinetic energies:

$$T = \frac{1}{2}m\boldsymbol{v}_\mathrm{CG} \cdot \boldsymbol{v}_\mathrm{CG} + \frac{1}{2}I_\mathrm{CG}\dot{\theta}^2,$$

where $\boldsymbol{v}_\mathrm{CG}$ is the velocity vector of the CG. The velocity vector of the CG is given by the vector sum of the heave velocity, which is evaluated at the EA, and of the tangential velocity due to pitch:

$$\boldsymbol{v}_\mathrm{CG} = \boldsymbol{v}_\mathrm{EA} + \boldsymbol{v}_t.$$

In terms of our generalized coordinates, the heave velocity at the EA is simply:

$$\boldsymbol{v}_\mathrm{EA} = \dot{h}\hat{\boldsymbol{i}}_2,$$

while the tangential velocity due to pitch is given by the cross product of the angular velocity vector and the position vector from the EA to the CG (remember the formula for the tangential velocity in rotational motion: $\boldsymbol{v} = \boldsymbol{\omega} \times \boldsymbol{r}$):

$$\boldsymbol{v}_t = \theta\hat{\boldsymbol{b}}_3 \times b\left((1+e) - (1+a)\right)\hat{\boldsymbol{b}}_1 = b\dot{\theta}(e - a)\hat{\boldsymbol{b}}_2.$$

The quantity $x_\theta = (e - a)$ represents the nondimensional distance (in terms of half-chord) between the EA and the CG and it is also referred to as the **static unbalance parameter**. By our definitions, $x_\theta$ is positive when the CG is ahead of the EA and negative when it is behind it.

The unit vector $\hat{\boldsymbol{b}}_2$ has components in the inertial frame given by $\cos\theta\hat{\boldsymbol{i}}_2$ and $-\sin\theta\hat{\boldsymbol{i}}_1$. Consequently, the velocity vector of the CG can be expressed in the inertial frame as:

$$\boldsymbol{v}_\mathrm{CG} = \dot{h}\hat{\boldsymbol{i}}_2 + b\dot{\theta}x_\theta(\cos\theta\hat{\boldsymbol{i}}_2 - \sin\theta\hat{\boldsymbol{i}}_1).$$

To simplify the expression, we use the small angles assumption, which we will use anyway to calculate our aerodynamic forces. If $\theta$ is small, then we can approximate $\cos\theta \approx 1$ and $\sin\theta \approx \theta$. Thus, the velocity vector of the center of mass simplifies to:

$$\boldsymbol{v}_\mathrm{CG} \approx \left(\dot{h} + b\dot{\theta}x_\theta\right)\hat{\boldsymbol{i}}_2 - b\dot{\theta}x_\theta\theta\hat{\boldsymbol{i}}_1.$$

The second term can be neglected in the assumption of small $\theta$, and consequently we can write the product $\boldsymbol{v}_\mathrm{CG} \cdot \boldsymbol{v}_\mathrm{CG}$ as:

$$\boldsymbol{v}_\mathrm{CG} \cdot \boldsymbol{v}_\mathrm{CG} \approx \left(\dot{h} + b\dot{\theta}x_\theta\right)^2 = \dot{h}^2 + 2bx_\theta\dot{h}\dot{\theta} + b^2x_\theta^2\dot{\theta}^2.$$

Substituting this expression into the kinetic energy equation, we obtain:

$$T = \frac{1}{2}m\left(\dot{h}^2 + 2bx_\theta\dot{h}\dot{\theta} + b^2x_\theta^2\dot{\theta}^2\right) + \frac{1}{2}I_\mathrm{CG}\dot{\theta}^2 = \frac{1}{2}m\left(\dot{h}^2 + 2bx_\theta\dot{h}\dot{\theta}\right) + \frac{1}{2}I_\mathrm{EA}\dot{\theta}^2,$$

where $I_\mathrm{EA} = I_\mathrm{CG} + mb^2x_\theta^2$ is the mass moment of inertia about the EA, obtained using the parallel axis theorem.

The Lagrangian $\mathscr{L} = T - V$ can now be expressed as:

$$\mathscr{L} = \frac{1}{2}m\left(\dot{h}^2 + 2bx_\theta\dot{h}\dot{\theta}\right) + \frac{1}{2}I_\mathrm{EA}\dot{\theta}^2 - \frac{1}{2}k_h h^2 - \frac{1}{2}k_\theta \theta^2.$$

We then take the partial derivatives of the Lagrangian with respect to the generalized coordinates and their time derivatives:

$$
\frac{\partial \mathscr{L}}{\partial h} = -k_h h, \quad \frac{\partial \mathscr{L}}{\partial \theta} = -k_\theta \theta,\\[1ex]
\frac{\partial \mathscr{L}}{\partial \dot{h}} = m\dot{h} + mbx_\theta \dot{\theta}, \quad \frac{\partial \mathscr{L}}{\partial \dot{\theta}} = mbx_\theta \dot{h} + I_\mathrm{EA} \dot{\theta}.
$$

Successively, we compute the time derivatives $\frac{\mathrm{d}}{\mathrm{d}t}\left(\frac{\partial \mathscr{L}}{\partial \dot{\boldsymbol{q}}}\right)$:

$$
\frac{\mathrm{d}}{\mathrm{d}t}\left(\frac{\partial \mathscr{L}}{\partial \dot{h}}\right) = m\ddot{h} + mbx_\theta \ddot{\theta},\\[1ex]
\frac{\mathrm{d}}{\mathrm{d}t}\left(\frac{\partial \mathscr{L}}{\partial \dot{\theta}}\right) = mbx_\theta \ddot{h} + I_\mathrm{EA} \ddot{\theta}.
$$

The generalized forces associated with the degrees of freedom $h$ and $\theta$ can be derived from the work done by the aerodynamic lift through a virtual displacement and a virtual rotation about its point of application, that is to say the AC. Mathematically, the generalized forces are defined as:

$$\boldsymbol{Q} = \frac{\partial (\delta W)}{\partial (\delta \boldsymbol{q})}.$$

In the assumption of quasi-steady aerodynamics, the lift force of the typical section depends on the angle of attack given by the pitch angle $\theta$ and by the effective angle of attack due to the plunge velocity $\dot{h}$:

$$
L = 2 q b c_{l\alpha} \left(\theta - \frac{\dot{h}}{V}\right),
$$

where $q = \frac{1}{2}\rho V^2$ is the dynamic pressure, $2b$ is the chord length, $c_{l\alpha} = 2\pi$ is the lift curve slope for a thin airfoil, and the minus sign in front of $\dot{h}/V$ accounts for the fact that a positive heave velocity reduces the effective angle of attack.

To find the work done by the lift, we need to compute the virtual displacement of the AC. Similarly to what we did for the CG, the velocity vector of the AC is given by the sum of the heave velocity at the EA and of the tangential velocity due to pitch:

$$
\boldsymbol{v}_\mathrm{AC} = \dot{h}\hat{\boldsymbol{i}}_2 + \dot{\theta}\hat{\boldsymbol{b}}_3 \times b\left((1+e) - \frac{1}{2}\right)\hat{\boldsymbol{b}}_1 = \dot{h}\hat{\boldsymbol{i}}_2 + b\dot{\theta}\left(\frac{1}{2} + e\right)\hat{\boldsymbol{b}}_2,
$$

and, analogously to the velocity vector of the CG, in the small angle assumption this expression simplifies to:

$$\boldsymbol{v}_\mathrm{AC} \approx \left(\dot{h} + b\dot{\theta}\left(\frac{1}{2} + e\right)\right)\hat{\boldsymbol{i}}_2.$$

The virtual displacement of the AC can be obtained by replacing the time derivatives of the generalized coordinates with their virtual displacements:

$$
\delta\boldsymbol{p}_\mathrm{AC} = \left(\delta h + b\delta\theta\left(\frac{1}{2} + e\right)\right)\hat{\boldsymbol{i}}_2.
$$

The work done by the aerodynamic lift through the virtual displacement of the AC is then given by:

$$
\delta W = L\hat{\boldsymbol{i}}_2 \cdot \delta\boldsymbol{p}_\mathrm{AC} = 2 q b c_{l\alpha} \left(\theta - \frac{\dot{h}}{V}\right)\left(\delta h + b\delta\theta\left(\frac{1}{2} + e\right)\right).
$$

Taking the partial derivatives with respect to $\delta h$ and $\delta \theta$, we find the generalized forces:

$$
Q_h = \frac{\partial (\delta W)}{\partial (\delta h)} = 2 q b c_{l\alpha} \left(\theta - \frac{\dot{h}}{V}\right) = L,\\[1ex]
Q_\theta = \frac{\partial (\delta W)}{\partial (\delta \theta)} = 2 q b c_{l\alpha} \left(\theta - \frac{\dot{h}}{V}\right) b\left(\frac{1}{2} + e\right) = L b\left(\frac{1}{2} + e\right).
$$

Substituting the expressions for the Lagrangian derivatives and the generalized forces into Lagrange's equations, we obtain the equations of motion for the typical section with pitch and plunge degrees of freedom:

$$
m\ddot{h} + mbx_\theta \ddot{\theta} + k_h h = 2 q b c_{l\alpha} \left(\theta - \frac{\dot{h}}{V}\right),\\[1ex]
I_\mathrm{EA} \ddot{\theta} + mbx_\theta \ddot{h} + k_\theta \theta = 2 q b^2 c_{l\alpha} \left(\frac{1}{2} + e\right)\left(\theta - \frac{\dot{h}}{V}\right).
$$

Let's stop for a moment and think about these equations. We can immediately make two observations:
- The equations are coupled through the inertial terms involving $mbx_\theta$. As we predicted intuitively, this means that even in the absence of aerodynamic forces, the heave and pitch motions influence each other due to the offset between the CG and the EA.
- The equations are also coupled through the aerodynamic terms on the right-hand side. This is the aeroelastic coupling that we have seen in previous notebooks, where the aerodynamic forces depend on the structural motion.

We also note that this is a system of two coupled second-order linear ordinary differential equations with constant coefficients and that we can rewrite it in homogeneous form by bringing all terms to the left-hand side, substituting $q = \frac{1}{2}\rho V^2$ and $c_{l\alpha} = 2\pi$ to simplify the expressions:

$$
m\ddot{h} + mbx_\theta \ddot{\theta} + 2\pi \rho b V \dot{h} - 2\pi \rho b V^2 \theta + k_h h = 0,\\[1ex]
I_\mathrm{EA} \ddot{\theta} + mbx_\theta \ddot{h} + 2\pi \rho b^2 V \left(\frac{1}{2} + e\right) \dot{h} - 2\pi \rho b^2 V^2 \left(\frac{1}{2} + e\right) \theta + k_\theta \theta = 0.
$$

Once again, let's stop for a moment and think about the structure of these equations. We can see that they have the general form of a coupled mass-spring-damper system, where:
- The terms involving $\ddot{h}$ and $\ddot{\theta}$ represent the inertial forces.
- The terms involving $\dot{h}$ represent the aerodynamic damping forces.
- The terms involving $h$ and $\theta$ represent the elastic forces, given by the structural stiffness and the aerodynamic stiffness.

We can express the system in matrix form as:

$$
\boldsymbol{M}\ddot{\boldsymbol{q}} + \boldsymbol{C}\dot{\boldsymbol{q}} + \boldsymbol{K}\boldsymbol{q} = \boldsymbol{0},
$$

where the mass matrix $\boldsymbol{M}$, the damping matrix $\boldsymbol{C}$, and the stiffness matrix $\boldsymbol{K}$ are given by:

$$
\boldsymbol{M} = \begin{bmatrix}
m & mbx_\theta \\
mbx_\theta & I_\mathrm{EA}
\end{bmatrix}, \quad
\boldsymbol{C} = \begin{bmatrix}
2\pi \rho b V & 0 \\
2\pi \rho b^2 V \left(\frac{1}{2} + e\right) & 0
\end{bmatrix}, \quad
\boldsymbol{K} = \begin{bmatrix}
k_h & -2\pi \rho b V^2 \\
0 & k_\theta -2\pi \rho b^2 V^2 \left(\frac{1}{2} + e\right)
\end{bmatrix}.
$$

**Important observations:**

1. Notice how the damping matrix $\boldsymbol{C}$ has **zeros in the second column**. This means there's no direct damping term proportional to $\dot{\theta}$ in either equation. This is a consequence of our quasi-steady aerodynamics assumption.

2. The stiffness matrix $\boldsymbol{K}$ contains one **off-diagonal term** that couples the heave and pitch motions aerodynamically.

3. Both the damping and stiffness matrices **depend on flight speed** $V$. This is the hallmark of aeroelastic systems: the dynamic characteristics change with operating conditions!

This matrix formulation is compact and elegant, but similarly to our last notebook, in order to solve it computationally and understand its stability, we need to transform it into state-space form. Let's do that next.

## State-Space Representation: Setting Up for Eigenvalue Analysis

We have a second-order system of differential equations. Analogously to our last notebook, to perform eigenvalue analysis and time integration, we need to convert it into a first-order system in state-space form in order to perform eigenvalue analysis and time integration.

### Defining the State Vector

We define a **state vector** $\boldsymbol{x}$ that includes both the displacements and velocities:

$$
\boldsymbol{x} = \begin{bmatrix} h \\ \theta \\ \dot{h} \\ \dot{\theta} \end{bmatrix}
$$

This choice means our state vector has **4 components** for a 2-DOF system. The first derivative of the state vector is:

$$
\dot{\boldsymbol{x}} = \begin{bmatrix} \dot{h} \\ \dot{\theta} \\ \ddot{h} \\ \ddot{\theta} \end{bmatrix}
$$

### Converting to State-Space Form

Starting from our matrix equation:

$$\boldsymbol{M}\ddot{\boldsymbol{q}} + \boldsymbol{C}\dot{\boldsymbol{q}} + \boldsymbol{K}\boldsymbol{q} = \boldsymbol{0}$$

We can solve for the accelerations:

$$\ddot{\boldsymbol{q}} = -\boldsymbol{M}^{-1}\boldsymbol{C}\dot{\boldsymbol{q}} - \boldsymbol{M}^{-1}\boldsymbol{K}\boldsymbol{q}$$

Now we can write the state-space form:

$$
\dot{\boldsymbol{x}} = \boldsymbol{A}\boldsymbol{x}
$$

where the **system matrix** $\boldsymbol{A}$ is:

$$
\boldsymbol{A} = \begin{bmatrix}
\boldsymbol{0}_{2\times 2} & \boldsymbol{I}_{2\times 2} \\
-\boldsymbol{M}^{-1}\boldsymbol{K} & -\boldsymbol{M}^{-1}\boldsymbol{C}
\end{bmatrix}
$$

This is a $4 \times 4$ matrix. The identity matrix $\boldsymbol{I}$ in the upper right block ensures that the first two components of $\dot{\boldsymbol{x}}$ correspond to the velocities $\dot{h}$ and $\dot{\theta}$.

### The Flutter Eigenvalue Problem

As we learnt in our last notebook, the stability of our system is determined by the **eigenvalues** of the matrix $\boldsymbol{A}$. For a 2-DOF system, we'll have **4 eigenvalues** that come in **2 complex conjugate pairs** (if the system is underdamped).

For each eigenvalue $\lambda = \lambda_r + i\lambda_i$:
- The **real part** $\lambda_r$ determines stability:
  - If $\lambda_r < 0$: the mode is **stable** (decays)
  - If $\lambda_r = 0$: the mode is **neutrally stable** (constant amplitude)
  - If $\lambda_r > 0$: the mode is **unstable** (grows exponentially)
- The **imaginary part** $\lambda_i$ represents the **oscillation frequency**

Flutter occurs when one of the eigenvalues crosses the imaginary axis (i.e., when its real part becomes positive). In other words, $\lambda_r=0$ represents the **flutter boundary**.

Let's now implement this in code and see what happens as we vary the flight speed! 🎯

Let's start by defining the properties of our typical section. For the moment we will consider a configuration with the EA ahead of the CG, so with $x_\theta < 0$. We will see later why.

In [ ]:
# Define the parameters of our typical section
# Geometrical properties
c = 1.6    # [m]
b = c / 2  # semi-chord [m]
a = 0.1    # distance from mid-chord to CG (as a fraction of the semi-chord)
e = -0.1   # distance from mid-chord to EA (as a fraction of the semi-chord)
x_theta = e - a  # static unbalance parameter

# Inertial properties
m = 200.0     # mass per unit span [kg/m]
I_EA = 30.0  # mass moment of inertia about the elastic axis per unit span [kg m^2/m]

# Elastic properties
k_h = 4e4  # heave stiffness [N/m]
k_theta = 1e5  # torsional spring stiffness [Nm/rad]

# Air density at sea level
rho = 1.225  # [kg/m^3]

Did we choose randomly these parameters? Not really. We've tried to replicate more or less what would be the first bending and torsion frequencies of an aircraft wing. We can do a quick check by calculating the natural frequencies of the mechanically uncoupled system in vacuum, i.e. an hypothetical configuration of our system where the the heave and pitch motions are independent and there are no aerodynamic forces. In this case, the natural frequencies are given by:

$$
\omega_h = \sqrt{\frac{k_h}{m}}, \quad \omega_\theta = \sqrt{\frac{k_\theta}{I_\mathrm{EA}}}.
$$

In [ ]:
omega_h = np.sqrt(k_h / m)  # heave natural frequency [rad/s]
omega_theta = np.sqrt(k_theta / I_EA)  # pitch natural frequency [rad/s]
print(f"Heave natural frequency: {omega_h:.1f} rad/s, {omega_h / (2 * np.pi):.2f} Hz")
print(f"Pitch natural frequency: {omega_theta:.1f} rad/s, {omega_theta / (2 * np.pi):.2f} Hz")

Indeed, the typical values of first bending and torsion frequencies of a commercial aircraft wing are in the order of 1-3 Hz and 6-10 Hz, respectively, although of course these values can vary significantly depending on the specific aircraft. Remember that these frequencies are NOT the actual frequencies of the coupled system, and we've just used them as a quick and dirty way to some parameters that can be related to representative bending and torsion frequencies.

Now let's create the function to build our system matrix, so that we can later use it to compute the eigenvalues at different flight speeds.

In [ ]:
def build_system_matrix(rho, velocity):
    """
    Build the system matrix for eigenvalue analysis.
    
    Parameters
    ----------
    rho : float
        Air density [kg/m^3]
    velocity : float
        Freestream velocity (V)
    
    Returns
    -------
    system_matrix : numpy array
        System matrix (4x4)
    """
    # Mass matrix (symmetric)
    M = np.array([
        [m,          m*b*x_theta],
        [m*b*x_theta, I_EA      ]
    ])
    
    # Damping matrix (note: only first column is non-zero due to quasi-steady assumption)
    C = np.array([
        [2*np.pi*rho*b*velocity,               0],
        [2*np.pi*rho*b**2*velocity*(0.5 + e),  0]
    ])
    
    # Stiffness matrix (note the negative off-diagonal terms)
    K = np.array([
        [k_h, -2*np.pi*rho*b*velocity**2                     ],
        [0,    k_theta - 2*np.pi*rho*b**2*velocity**2*(0.5 + e)]
    ])

    # Invert the mass matrix
    M_inv = np.linalg.inv(M)
    
    # Build the state matrix in block form
    zeros = np.zeros((2, 2))
    identity = np.eye(2)
    
    # Top row blocks: [0, I]
    top = np.hstack([zeros, identity])
    
    # Bottom row blocks: [-M^(-1)K, -M^(-1)C]
    bottom = np.hstack([-M_inv @ K, -M_inv @ C])
    
    # Stack vertically
    system_matrix = np.vstack([top, bottom])
    
    # Return the system matrix
    return system_matrix

## Root Locus Analysis: Finding the Flutter Speed

Similarly to our last notebook, we compute the eigenvalues over a range of flight speeds and see how they evolve in the complex plane. Remember: the left half of the complex plane (negative real part) corresponds to stable modes, while the right half (positive real part) corresponds to unstable modes. Let's sweep a range of flight speeds between 0 and 150 m/s and see what happens!

In [ ]:
# Range of velocities to analyze
velocities = np.linspace(1, 150, 200)

# Initialize arrays to store real and imaginary parts of eigenvalues
# To plot the root locus, we will use the pyplot function scatter, which expects 1D arrays
# We know there will be 4 eigenvalues for each velocity, so we preallocate a 1D array with shape (N_velocities * 4,)
N_EIGENVALUES = 4
real_parts = np.zeros((len(velocities) * N_EIGENVALUES,))
imag_parts = np.zeros((len(velocities) * N_EIGENVALUES,))

# Iterate over velocities
for idx, V in enumerate(velocities):

    # Build system matrix
    A = build_system_matrix(rho, V)

    # Compute eigenvalues
    eigenvalues = np.linalg.eigvals(A)

    # Store real and imaginary parts
    # Since we have 4 eigenvalues for each velocity, we store them in the appropriate slice of the preallocated arrays
    real_parts[idx * N_EIGENVALUES : (idx + 1) * N_EIGENVALUES] = eigenvalues.real
    imag_parts[idx * N_EIGENVALUES : (idx + 1) * N_EIGENVALUES] = eigenvalues.imag

# Create figure
plt.figure()

# Scatter plot: x=Real, y=Imaginary, Color=Speed
# The arrays including the real and imaginary parts have shape (N_velocities * 4,)
# In order to color the points according to velocity, we need to repeat each velocity four times to match the structure of the real_parts and imag_parts arrays
# We use the repeat function from NumPy to achieve this
# The function call np.repeat(velocities, 4) creates an array where each element of the velocity array is repeated four consecutively
# The scatter function returns a PathCollection object that we can use to create the colorbar
sc = plt.scatter(real_parts, imag_parts, c=np.repeat(velocities, N_EIGENVALUES))

# Set plot elements
plt.xlabel("$\\mathbb{Re}(\\lambda)$ - Exponential Decay/Growth Rate")
plt.ylabel("$\\mathbb{Im}(\\lambda)$ - Oscillation Frequency")
plt.grid(True)

# Add colorbar and show plot
plt.colorbar(sc, label="$V$, m/s")
plt.show()

**Stop and Think.** Let's observe a few things from this root locus plot:
- We now have 4 eigenvalue trajectories, corresponding to 4 eigenvalues of our 2-DOF system.
- The eigenvalues appear in two complex conjugate pairs, as expected for an underdamped 2-DOF system.
- Each pair corresponds to a vibration mode of the system. The mode with lower frequency corresponds to the heave-dominated motion, while the mode with higher frequency corresponds to the pitch-dominated motion.
- Both modes start close to the imaginary axis at low speeds, as aerodynamic forces are the only source of damping in our system, and with no aerodynamics the system is undamped.
- The real part of the heave-dominated mode becomes more negative with increasing speed, indicating that it becomes more stable, while its frequency remains relatively constant (it increases only slightly).
- The pitch dominated mode, on the other hand, shows a different behavior as speed increases. Initially the real part becomes more negative, indicating increased stability, but at a certain speed, it starts to move towards the right half of the complex plane, until it crosses into the unstable region. The speed at which this happens is the flutter speed of the system, and the corresponding imaginary part gives us the flutter frequency. The frequency of the pitch-dominated mode appears to decrease with increasing speed, showing a more pronounced effect of aerodynamic stiffness compared to the heave-dominated mode.

### V-g and V-f Diagrams

Similarly to our previous notebook, we are now going to use the two companion plots of the root locus diagram:

1. **V-g diagram**: Damping ratio vs. velocity
   - Shows how stable/unstable each mode is
   - **Flutter speed** = where damping crosses zero

2. **V-f diagram**: Frequency vs. velocity
   - Shows how mode frequencies change with speed
   - **Flutter frequency** = frequency at flutter speed

In [ ]:
# Initialize arrays to store frequency and damping ratio
# Since we have two degree of freedom, there will be two frequencies and two damping ratios per velocity
frequencies = np.zeros((len(velocities), 2))  # these will be angular frequencies omega
damping_ratios = np.zeros((len(velocities), 2))

# We reshape the arrays of the real and imaginary parts to a shape (N_velocities, N_EIGENVALUES)
real_parts = real_parts.reshape((len(velocities), N_EIGENVALUES))
imag_parts = imag_parts.reshape((len(velocities), N_EIGENVALUES))

# We iterate over the velocities to calculate frequency and damping ratio
for idx in range(len(velocities)):

    # Sort by highest imaginary part (frequency)
    sort_idx = np.argsort(imag_parts[idx])[::-1]  # descending order

    # Keep only the eigenvalues with the two highest frequencies (positive frequencies)
    sorted_real = real_parts[idx][sort_idx[:2]]
    sorted_imag = imag_parts[idx][sort_idx[:2]]

    # Take the first two eigenvalues (positive frequencies)
    frequencies[idx] = np.abs(sorted_imag)
    
    # Calculate Damping Ratio zeta
    damping_ratios[idx] = -sorted_real / np.sqrt(sorted_real**2 + sorted_imag**2)

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

# Plot 1: V-g plot (damping vs velocity)
ax1.plot(velocities, damping_ratios)
ax1.set_ylabel('$\\zeta$')
ax1.grid(True)

# Plot 2: V-f plot (frequency vs velocity)
ax2.plot(velocities, frequencies)
ax2.set_ylabel('$\\omega$, rad/s')
ax2.set_xlabel('$V$, m/s')
ax2.grid(True)

# Set tigt layout and show plot
ax1.legend(['Pitch-dominated mode', 'Heave-dominated mode'])
plt.tight_layout()
plt.show()

**Interpreting the V-g Diagram:**

- **Positive damping** ($\zeta$ > 0): Mode is stable, oscillations decay
- **Zero damping** ($\zeta$ = 0): Neutral stability, constant amplitude oscillations
- **Negative damping** ($\zeta$ < 0): Mode is unstable, oscillations grow ⚠️

The flutter speed $V_F$ is where the damping curve crosses zero from above.

**Interpreting the V-f Diagram:**

- Shows how natural frequencies change with airspeed
- Modes may **approach each other** as speed increases
- When modes get close in frequency, they can **couple** and lead to flutter
- The flutter frequency $\omega_F$ is the frequency at which flutter occurs

## Time-Domain Simulations: Seeing Flutter in Action

Eigenvalue analysis tells us *when* flutter occurs, but to really **see** what flutter looks like, we need to simulate the time response of our system. In order to do this, we need to set up the function that computes the time derivative of the state vector, which we can then integrate over time using `odeint`, like we did in our last notebook.

In [ ]:
# Define the function that returns dx/dt = A * x
def state_space_model(x, time_array, system_matrix):
    """
    Computes the derivative of the state vector.

    Parameters:
    x (array): State vector [h, theta, h_dot, theta_dot]
    time_array (float): Time (not used here because our system is linear time-invariant, but odeint requires it)
    system_matrix (matrix): System matrix

    Returns:
    dxdt (array): Time derivative of the state vector
    """
    dxdt = system_matrix @ x
    return dxdt

Let's now pick three representative flight speeds:

1. **Well below flutter speed**: 60 m/s, we expect a stable, well-damped motion
2. **Just below flutter speed**: 120 m/s, we expect lightly damped, slow decaying motion
3. **Above flutter speed**: 140 m/s, we expect unstable, exponentially growing oscillations ⚠️

We'll give the system a small initial disturbance and watch what happens. Careful: this time we have two degrees of freedom, so we need to set initial conditions for both heave and pitch and we will have to plot both responses!

In [ ]:
# Define flight speeds to test
flight_speeds = [60, 120, 130]

# Define simulation parameters
t = np.linspace(0, 3, 1000)  # Simulate for 3 seconds
initial_condition = [0.5, 0.0, 0.0, 0.0]  # Initial displacement h = 0.5 m, theta = 0 rad, h_dot = 0 m/s, theta_dot = 0 rad/s

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

# Iterate over speeds
for V in flight_speeds:

    # Build the system matrix for this speed
    A = build_system_matrix(rho, V)

    # Solve the ODE
    solution_array = odeint(state_space_model, initial_condition, t, args=(A,))

    # Extract 'h' (the first column of the solution)
    h_response = solution_array[:, 0]
    theta_response = solution_array[:, 1]

    # Plot
    ax1.plot(t, h_response, label=f"$V = {V}$ m/s")
    ax2.plot(t, np.degrees(theta_response), label=f"$V = {V}$ m/s")

# Set axes labels, legend, and grid
ax2.set_xlabel("$t$, s")
ax1.set_ylabel("$h$, m")
ax2.set_ylabel("$\\theta$, deg")
ax2.legend()
ax1.grid(True)
ax2.grid(True)
plt.show()

What do you see? As usual, let's make some observations.
- At 60 m/s, both heave and pitch motions show damped oscillations that decay to zero over time, indicating a stable system. As predicted by the V-g plot, the damping of the pitch-dominated mode is lower than that of the heave-dominated mode, resulting in a slower decay of the pitch motion.
- At 120 m/s, the heave motion decays even faster than at 60 m/s. Indeed the damping of the heave-dominated mode at this speed is higher. The pitch motion, on the other hand, shows almost no decay, indicating that the system is close to neutral stability. This is consistent with the V-g plot, where the damping of the pitch-dominated mode is very close to zero at this speed.
- At 130 m/s, both heave and pitch motions grow exponentially over time, indicating an unstable system. The pitch motion grows faster than the heave motion, which is consistent with the fact that the pitch-dominated mode has become unstable at this speed, as shown in the V-g plot.

## The Physical Mechanism of Flutter 🌀

Before we move to exercises, let's take a moment to understand **why** flutter happens. It's not just mathematics, there's an interesting physical mechanism at work.

For the case of classical bending-torsion flutter this mechanism is well illustrated in the figure below.

<figure>
<img src="../figures/03_demasi_flutter.png" style="width:100%">
<figcaption align = "center"> Illustration of the physical mechanism behind classical bending-torsion flutter. Reproduced from "Introduction to Unsteady Aerodynamics and Dynamic Aeroelasticity", Luciano Demasi, Springer (2024). </figcaption>
</figure>

1. **Frequency Coalescence**: The heave and pitch mode natural frequencies become close to each other
   - Look at your V-f diagram: do the frequencies approach each other?

2. **Phase Relationship**: The heave and pitch motions develop a specific phase relationship
   - When the wing pitches nose-up, it's also moving upward
   - This creates an effective angle of attack that **reinforces** both motions

3. **Energy Transfer**: The coupled motion extracts energy from the airflow
   - The aerodynamic forces do **positive work** on the system
   - This compensates for (and eventually exceeds) structural damping

4. **Self-Excitation**: The system becomes a self-excited oscillator
   - The amplitude grows until limited by nonlinear effects or structural failure

### Why Quasi-Steady Flutter is Different

The flutter you've just analyzed uses **quasi-steady aerodynamics**, which assumes aerodynamic forces respond instantaneously to motion. This is a simplification that:

- **Misses** some physical mechanisms (like added mass and aerodynamic lag effects)
- **Inaccurately predicts** flutter speed (real flutter might occur at lower speeds)
- **Still captures** the essential coupling mechanism and basic behavior

For more accurate predictions, especially for high-frequency oscillations, we need **unsteady aerodynamics**, but that's a topic for future notebooks! 😊

## 🎯 Exercise: Exploring Design Parameter Effects on Flutter

Now it's your turn to explore! Flutter speed is sensitive to many design parameters. Understanding these sensitivities is crucial for aircraft design.

### Your Tasks:

1. **Relative position between CG and Elastic Axis**: 
   - Modify the either the EA position parameter $e$ or the CG position parameter $a$ such that the CG moves closer to the EA (increase $x_\theta$ from its current negative value)
   - Keep all other parameters the same
   - How does this affect the flutter speed?
   - What happens when the CG is exactly at the EA? And when the CG is ahead of the EA?

2. **Frequency Ratio Investigation**:
   - Calculate the ratio of uncoupled natural frequencies $\omega_\theta / \omega_h$
   - Play with this ratio making it larger or smaller by adjusting stiffnesses
   - What happens to the flutter speed when the uncoupled frequencies are close vs. when they are far apart?

### Suggested Approach:

- Set up your code cell to run the eigenvalue analysis for a range of velocities and to plot the root locus diagram
- Select the parameter you want to investigate
- Try a few values and see how the root locus changes
- Let's discuss what you see!

In [ ]:
# Your code here! 🚀
# Copy the code for the eigenvalue analysis at different speeds
# Select parameter to investigate
# Run multiple times and see how the root locus diagram changes

## Summary 🎓

Congratulations! You've just completed your introduction to flutter analysis. Let's recap what you've learned:

### Key Takeaways:

1. **Coupling is Essential**: Flutter requires coupling between lift and both the heave and pitch motions

2. **State-Space Formulation**: Converting 2nd-order ODEs to 1st-order form
   - Enables eigenvalue analysis
   - 4 eigenvalues for a 2-DOF system (2 complex conjugate pairs)

3. **Flutter Criteria**: Instability when eigenvalue real part becomes positive
   - Root locus shows eigenvalue trajectories
   - V-g diagram shows damping evolution
   - V-f diagram shows frequency evolution

4. **Physical Mechanism**: Self-excited oscillation extracting energy from airflow
   - Frequency coalescence + phase relationship
   - Aerodynamic work exceeds structural damping

5. **Design Parameters Matter**: Flutter speed depends on:
   - Mass distribution (CG and EA positions)
   - Structural stiffnesses and frequency ratios

### What We Haven't Covered:

- **Unsteady Aerodynamics**: More accurate modeling of aerodynamic forces
  - Theodorsen's function for oscillating airfoils
  - Wagner's function for arbitrary motion
  - Finite-state approximations (e.g., Roger's approximation)

- **Multiple Degrees of Freedom**: Real aircraft wings
  - Finite element models
  - Modal analysis and reduction

- **Nonlinear Effects**: What happens after flutter onset
  - Limit cycle oscillations
  - Structural nonlinearities

But you now have the fundamental tools to understand these more advanced topics in the future!

### Final Thought 💭

Aeroelastic instabilities were one of the major challenges in the early days of aviation. The understanding you've gained today represents decades of research that enabled safe high-speed flight. Every time you fly, aeroelastic analysis like this has ensured your safety.

**Well done!** 🎉